In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 공통설정

In [2]:
import os
import json
import glob
import pandas as pd
from datasets import Dataset
from dotenv import load_dotenv
import gc
load_dotenv()


from openai import OpenAI
from ragas.llms import llm_factory
from langchain_upstage import UpstageEmbeddings


openai_client = OpenAI()
ragas_llm = llm_factory(
    model="gpt-4o-mini",
    provider="openai",
    client=openai_client,
)


# 평가용 임베딩 (Upstage)
ragas_embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)


print("✅ ragas_llm ready:", type(ragas_llm))
print("✅ ragas_embeddings ready:", type(ragas_embeddings))


# embedding 테스트
test_vec = ragas_embeddings.embed_query("테스트")
print(f"✅ Embedding test successful! Dimension: {len(test_vec)}")

✅ ragas_llm ready: <class 'ragas.llms.base.InstructorLLM'>
✅ ragas_embeddings ready: <class 'langchain_upstage.embeddings.UpstageEmbeddings'>
✅ Embedding test successful! Dimension: 4096


# 2. must-have 함수실행

In [3]:
import re
from typing import Dict, Any, List, Tuple, Optional


# -----------------
# must-have rules
# -----------------


def load_must_have_rules(path: str) -> Dict[int, Dict[str, Any]]:
    if not os.path.exists(path):
        print(f"⚠️ must_have_rules.json 없음 → must-have 체크 스킵: {path}")
        return {}
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)
    return {int(k): v for k, v in raw.items() if str(k).isdigit()}





def check_must_have(
    qid: int,
    retrieved_articles: List[str],
    retrieved_contexts: List[str],
    rules: Optional[Dict[int, Dict[str, Any]]] = None
) -> Tuple[Optional[bool], str]:
    if not rules or qid not in rules:
        return None, "no_rule"


    rule = rules[qid]


    for a in rule.get("must_have_any", []):
        if a in retrieved_articles:
            return True, f"hit_article:{a}"


    joined = "\n".join(retrieved_contexts or [])
    for pat in rule.get("must_have_regex_any", []):
        if pat in joined:
            return True, f"hit_keyword:{pat}"


    return False, "miss_all"




# -----------------
# 조문번호 추출
# -----------------
_ARTICLE_RE = re.compile(r"제\s*\d+\s*조(?:\s*의\s*\d+)?")


def extract_article_numbers(text: str, max_unique: int = 30) -> List[str]:
    hits = _ARTICLE_RE.findall(text or "")
    out, seen = [], set()
    for h in hits:
        h = re.sub(r"\s+", "", h)
        if h not in seen:
            seen.add(h)
            out.append(h)
        if len(out) >= max_unique:
            break
    return out


def extract_articles_from_doc(doc) -> List[str]:
    md = getattr(doc, "metadata", {}) or {}
    for k in ["article", "조문", "조문번호"]:
        if k in md:
            return extract_article_numbers(str(md[k]))
    return extract_article_numbers(getattr(doc, "page_content", ""))

# 3. RAGAS 평가

In [4]:
import os
import re
import sys
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any

from docx import Document as DocxDocument
from datasets import Dataset

from ragas import evaluate
from ragas.metrics._context_precision import ContextPrecision
from ragas.metrics._context_recall import ContextRecall

import plotly.express as px
import plotly.graph_objects as go

# -----------------------------
# 0) RAG 모듈 import (프로젝트 구조 그대로)
# -----------------------------
MODULES_PATH = os.path.join(os.getcwd(), "chatbot_app", "modules")
sys.path.append(MODULES_PATH)
sys.path.append(os.getcwd())
from chatbot_app.modules.rag_module import create_pipeline, RAGConfig

# -----------------------------
# 1) 입력: 질문 docx 경로
# -----------------------------
DOC_PATH = "./data/RAGAS/RAGAS_질문_25개_정리본.docx"

def parse_ragas_docx(doc_path: str) -> List[Dict[str, Any]]:
    doc = DocxDocument(doc_path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
    text = "\n".join(paras)

    blocks = re.split(r"\n(?=\d+\.\s)", text)

    items = []
    for b in blocks:
        b = b.strip()
        if not re.match(r"^\d+\.\s", b):
            continue

        parts = b.split("✔️ 모범답안", 1)
        if len(parts) != 2:
            continue

        q_part = re.sub(r"^\d+\.\s*", "", parts[0].strip())
        a_part = parts[1].strip()

        items.append({"question": q_part.strip(), "ground_truth": a_part.strip()})

    if len(items) == 0:
        raise ValueError("docx에서 문항을 1개도 파싱하지 못했습니다. 문서 포맷을 확인하세요.")
    return items

items = parse_ragas_docx(DOC_PATH)
print(f"✅ 파싱된 문항 수: {len(items)} (기대: 25)")

# -----------------------------
# 2) run_tag 생성 (파일명 식별용)
# -----------------------------
def make_run_tag(cfg: RAGConfig) -> str:
    rerank_flag = "T" if getattr(cfg, "enable_rerank", False) else "F"
    dw = getattr(cfg, "hybrid_dense_weight", None)
    sw = getattr(cfg, "hybrid_sparse_weight", None)
    k1 = getattr(cfg, "bm25_k1", None)
    dc = getattr(cfg, "bm25_max_doc_chars", None)
    return (
        f"s{cfg.search_multiplier}"
        f"_law{cfg.k_law}_rule{cfg.k_rule}_case{cfg.k_case}"
        f"_dw{dw}_sw{sw}_k1{k1}_dc{dc}"
        f"_rerank{rerank_flag}"
    )

# -----------------------------
# 3) 저장 루트 (요구사항: ./data/RAGAS/ 밑)
# -----------------------------
OUT_DIR = "./data/RAGAS"
RUNS_ROOT = os.path.join(OUT_DIR, "runs")
os.makedirs(RUNS_ROOT, exist_ok=True)

# -----------------------------
# 4) R_i 가중치 (요구사항대로)
# -----------------------------
W_RECALL = 0.6
W_PREC   = 0.4

# -----------------------------
# 5) 캐시(jsonl) 생성 + RAGAS 평가 1회(run_once)
# -----------------------------
def run_once(cfg: RAGConfig, items: List[Dict[str, Any]], run_idx: int) -> Dict[str, Any]:
    run_tag = make_run_tag(cfg)
    run_dir = os.path.join(RUNS_ROOT, run_tag, f"run_{run_idx:02d}")
    os.makedirs(run_dir, exist_ok=True)

    cache_jsonl = os.path.join(run_dir, "rag_run_cache.jsonl")

    # (1) pipeline 생성
    pipeline = create_pipeline(config=cfg)

    # (2) 캐시 저장 (jsonl)
    with open(cache_jsonl, "w", encoding="utf-8") as f:
        for i, ex in enumerate(items, start=1):
            q = ex["question"]
            gt = ex["ground_truth"]

            
            print(f"[run_{run_idx:02d}] Q{i} chars=", len(q))
            if len(q) > 2000:
                print("[WARN] very long question head:", q[:200])

            MAX_QUERY_CHARS = 2000  # 안전값(너무 보수적이면 1500~2000까지 올려도 됨)

            q_for_retrieval = q if len(q) <= MAX_QUERY_CHARS else q[:MAX_QUERY_CHARS]
            
            try:
                trace = pipeline.answer_with_trace(q_for_retrieval, skip_normalization=False)
            except Exception as e:
                print(f"[ERROR] run_{run_idx:02d} Q{i} retrieval failed:", repr(e))
                trace = {"answer": "", "docs": [], "normalized_query": ""}

            answer = trace.get("answer", "") or ""
            docs = trace.get("docs", []) or []

            contexts = []
            for d in docs:
                try:
                    contexts.append(d.page_content)
                except Exception:
                    pass

            row = {
                "id": i,
                "question": q,
                "answer": answer,
                "contexts": contexts,
                "reference": gt,
                "ground_truths": [gt],
                "normalized_query": trace.get("normalized_query", ""),
            }
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    # (3) ragas용 rows 생성 (ragas공통.txt 스키마를 그대로 사용)
    raw_rows = []
    with open(cache_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                raw_rows.append(json.loads(line))

    # ragas공통.txt에 있는 normalize_row_schema()를 그대로 쓰는 대신,
    # 동일한 컬럼키(question/answer/contexts/reference)를 이미 넣었으므로 최소 정리만 수행
    rows_local = []
    for r in raw_rows:
        rr = dict(r)
        if not rr.get("question"):
            continue
        if not rr.get("answer"):
            rr["answer"] = "답변을 생성할 수 없습니다."
        if "contexts" not in rr or rr["contexts"] is None:
            rr["contexts"] = []
        if not isinstance(rr["contexts"], list):
            rr["contexts"] = [str(rr["contexts"])]
        if "reference" not in rr:
            rr["reference"] = ""
        rows_local.append(rr)

    # 데이터 검증(함수는 ragas공통.txt에서 제공)
    validate_data(rows_local, required_fields=["question", "contexts"])

    # ✅ 중요: context_precision / context_recall 입력 클립을 분리
    rows_cp = clip_rows(rows_local, max_ctx=3, max_chars=600)
    rows_cr = clip_rows(
        rows_local,
        max_ctx=2,
        max_chars=200,
        clip_reference_chars=300,
        clip_answer_chars=300,
    )

    ds_p = Dataset.from_list([
        {
            "question": r["question"],
            "answer": r["answer"],
            "contexts": r.get("contexts", []),
            "reference": r.get("reference", ""),
        }
        for r in rows_cp
    ])

    ds_r = Dataset.from_list([
        {
            "question": r["question"],
            "answer": r["answer"],
            "contexts": r.get("contexts", []),
            "reference": r.get("reference", ""),
        }
        for r in rows_cr
    ])

# (4) RAGAS 평가: precision / recall
    res_p = evaluate(
        dataset=ds_p,
        metrics=[ContextPrecision()],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    df_p = res_p.to_pandas()
    p_csv = os.path.join(run_dir, "ragas_result_context_precision.csv")
    df_p.to_csv(p_csv, index=False, encoding="utf-8")

    res_r = evaluate(
        dataset=ds_r,
        metrics=[ContextRecall()],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    df_r = res_r.to_pandas()
    r_csv = os.path.join(run_dir, "ragas_result_context_recall.csv")
    df_r.to_csv(r_csv, index=False, encoding="utf-8")

    # (5) per-question merge → R_i (행: 1~10)
    # id 컬럼이 없다면 보정
    if "id" not in df_p.columns:
        df_p = df_p.copy()
        df_p["id"] = range(1, len(df_p) + 1)
    if "id" not in df_r.columns:
        df_r = df_r.copy()
        df_r["id"] = range(1, len(df_r) + 1)

    dfm = pd.merge(
        df_r[["id", "context_recall"]],
        df_p[["id", "context_precision"]],
        on="id",
        how="inner",
    )
    dfm["R_i"] = (W_RECALL * dfm["context_recall"].astype(float) + W_PREC * dfm["context_precision"].astype(float))

    # 저장용 2자리
    dfm_save = dfm.copy()
    for c in ["context_recall", "context_precision", "R_i"]:
        dfm_save[c] = dfm_save[c].astype(float).round(2)

    perq_csv = os.path.join(run_dir, "per_question.csv")
    dfm_save.to_csv(perq_csv, index=False, encoding="utf-8")

    cleanup_memory()
    
    print(f"[run{run_idx}] Q{i} q_len={len(q)} retrieval_len={len(q_for_retrieval)}")
    return {
        "run_tag": run_tag,
        "run_idx": run_idx,
        "run_dir": run_dir,
        "cache_jsonl": cache_jsonl,
        "precision_csv": p_csv,
        "recall_csv": r_csv,
        "per_question_csv": perq_csv,
        "df_perq": dfm_save,
        "R_mean": float(dfm["R_i"].mean()),
        "R_std": float(dfm["R_i"].std(ddof=1)) if len(dfm) > 1 else 0.0,
    }
    


# -----------------------------
# 6) 반복 실행(run_repeat): cfg 튜닝용
# -----------------------------
def run_repeat(cfg: RAGConfig, items: List[Dict[str, Any]], n: int = 3) -> List[Dict[str, Any]]:
    results = []
    for run_idx in range(1, n + 1):
        print(f"\n==============================")
        print(f"RUN {run_idx:02d}/{n} | {make_run_tag(cfg)}")
        print(f"==============================")
        results.append(run_once(cfg, items, run_idx))
    return results

# -----------------------------
# 7) wide CSV 생성 (행: 1~10, 열: r1-precision/r1-recall/r1-Ri/...)
# -----------------------------
def build_wide_csv(run_tag: str, results: List[Dict[str, Any]]) -> str:
    """질문별 wide 테이블 + 마지막 mean 행 + Ri-mean(3회 평균) 컬럼 생성"""
    base_dir = os.path.join(RUNS_ROOT, run_tag)
    ids = pd.DataFrame({"id": range(1, n_questions + 1)})
    wide = ids.copy()

    # run별 병합
    for res in results:
        ridx = res["run_idx"]
        dfm = res["df_perq"][["id", "context_precision", "context_recall", "R_i"]].copy()
        dfm = dfm.rename(columns={
            "context_precision": f"r{ridx}-precision",
            "context_recall": f"r{ridx}-recall",
            "R_i": f"r{ridx}-Ri",
        })
        wide = pd.merge(wide, dfm, on="id", how="left")

    wide = wide.set_index("id").sort_index()

    # 질문별 Ri 평균(3회)
    ri_cols = [c for c in wide.columns if c.endswith("-Ri")]
    wide["Ri-mean"] = wide[ri_cols].mean(axis=1, skipna=True)

    # 마지막 mean 행(열별 평균)
    mean_row = wide.mean(axis=0, skipna=True).to_frame().T
    mean_row.index = ["mean"]
    wide2 = pd.concat([wide, mean_row], axis=0)

    out_csv = os.path.join(base_dir, "runs_wide_3repeats.csv")
    wide2.to_csv(out_csv, encoding="utf-8")
    print("✅ saved:", out_csv)

    # 보기 좋게 소수 2자리로 출력(저장은 float 그대로)
    display(wide2.round(2))
    return out_csv


# -----------------------------
# 8) 3회 평균 기반 이미지(평균만) 저장
# -----------------------------
def save_png(fig, path: str):
    fig.write_image(path, scale=2)
    print("🖼️ saved:", path)

def make_mean_images(run_tag: str, results: List[Dict[str, Any]]):
    """3회 결과를 질문별 평균으로 합쳐서 bar/table PNG 1세트만 저장"""
    base_dir = os.path.join(RUNS_ROOT, run_tag)
    out_dir = os.path.join(base_dir, "mean")
    os.makedirs(out_dir, exist_ok=True)

    # results -> (id, recall/precision/R_i) 평균
    dfs = [r["df_perq"][["id", "context_recall", "context_precision", "R_i"]].copy() for r in results]
    merged = dfs[0].rename(columns={
        "context_recall": "r1_recall",
        "context_precision": "r1_precision",
        "R_i": "r1_Ri",
    })

    # 안전하게 run_idx 기준으로 붙이기
    by_id = pd.DataFrame({"id": range(1, n_questions + 1)})
    for r in results:
        ridx = r["run_idx"]
        df = r["df_perq"][["id", "context_recall", "context_precision", "R_i"]].copy()
        df = df.rename(columns={
            "context_recall": f"r{ridx}_recall",
            "context_precision": f"r{ridx}_precision",
            "R_i": f"r{ridx}_Ri",
        })
        by_id = by_id.merge(df, on="id", how="left")

    recall_cols = [c for c in by_id.columns if c.endswith("_recall")]
    prec_cols = [c for c in by_id.columns if c.endswith("_precision")]
    ri_cols = [c for c in by_id.columns if c.endswith("_Ri")]

    mean_df = pd.DataFrame({
        "id": by_id["id"],
        "context_recall": by_id[recall_cols].mean(axis=1, skipna=True),
        "context_precision": by_id[prec_cols].mean(axis=1, skipna=True),
        "R_i": by_id[ri_cols].mean(axis=1, skipna=True),
    }).sort_values("id")

    mean_df["label"] = mean_df["id"].astype(str)

    # bar (고정순서 Q1->Q10)
    fig_bar = px.bar(
        mean_df,
        x="R_i",
        y="label",
        orientation="h",
        title=f"R_i per Question (mean of 3 runs, fixed order) | {run_tag}",
        text="R_i",
    )
    fig_bar.update_traces(texttemplate="%{text:.2f}", textposition="outside")
    fig_bar.update_layout(
        xaxis=dict(range=[0, 1.05]),
        yaxis=dict(autorange="reversed"),
        height=520
    )

    # table (소수 2자리 문자열로 표시)
    tdf = mean_df[["id", "context_recall", "context_precision", "R_i"]].copy()
    for c in ["context_recall", "context_precision", "R_i"]:
        tdf[c] = tdf[c].map(lambda x: f"{float(x):.2f}")

    fig_table = go.Figure(
        data=[go.Table(
            header=dict(values=list(tdf.columns)),
            cells=dict(values=[tdf[c].tolist() for c in tdf.columns]),
        )]
    )
    fig_table.update_layout(title="Per-question metrics (mean of 3 runs, 2 decimals)", height=420)

    bar_path = os.path.join(out_dir, "mean_runs_bar.png")
    tbl_path = os.path.join(out_dir, "mean_runs_table.png")
    save_png(fig_bar, bar_path)
    save_png(fig_table, tbl_path)
    return bar_path, tbl_path
must_rules = load_must_have_rules("./data/RAGAS/must_have_rules.json")

✅ 파싱된 문항 수: 25 (기대: 25)
⚠️ must_have_rules.json 없음 → must-have 체크 스킵: ./data/RAGAS/must_have_rules.json


# 4. 튜닝

In [5]:
%%time
cfg = RAGConfig(
    k_law=7,             # 5
    k_rule=7,            # 5
    k_case=3,            # 3
    search_multiplier=4, # 2
    enable_rerank=False,
    hybrid_dense_weight=0.5,  # 0.6
    hybrid_sparse_weight=0.5, # 0.4
    bm25_k1=1.5,              # 1.5
    bm25_max_doc_chars=3000,  # 3000
)


results = run_repeat(cfg, items, n=2)

2026-02-03 17:00:55,651 - chatbot_app.modules.rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...



RUN 01/2 | s4_law7_rule7_case3_dw0.5_sw0.5_k11.5_dc3000_rerankF


2026-02-03 17:00:58,553 - chatbot_app.modules.rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-02-03 17:00:58,601 - chatbot_app.modules.rag_module - INFO - ℹ️ SimpleTokenizer 사용 (BM25)


[run_01] Q1 chars= 32


2026-02-03 17:00:59,237 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-03 17:00:59,248 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 전입신고(주민등록)·확정일자(확정일자) 했는데, 확정일자부(확정일자) 내용까지 중요한가요?
2026-02-03 17:00:59,250 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='전입신고(주민등록)·확정일자(확정일자) 했는데, 확정일자부(확정일자) 내용까지 중요한가요?'
2026-02-03 17:00:59,863 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-03 17:01:01,940 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-03 17:01:03,967 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-03 17:01:06,264 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-03 17:01:06,951 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 20

...

[run_01] Q24 chars= 72


2026-02-03 17:12:00,457 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-03 17:12:00,466 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 다가구주택(임차주택)에 전세(임대차보증금)로 들어갔는데 저보다 나중에 들어온 세입자(임차인)들 때문에 경매(경매절차)에서 제 배당금(우선변제권)이 줄어들었습니다. 왜 이런 일이 생기나요?
2026-02-03 17:12:00,467 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='다가구주택(임차주택)에 전세(임대차보증금)로 들어갔는데 저보다 나중에 들어온 세입자(임차인)들 때문에 경매(경매절차)에서 제 배당금(우선변제권)이 줄어들었습니다. 왜 이런 일이 생기나요?'
2026-02-03 17:12:01,404 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-03 17:12:02,158 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-03 17:12:02,988 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-03 17:12:03,961 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02

[run_01] Q25 chars= 66


2026-02-03 17:12:33,763 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-03 17:12:33,763 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차 매각기일이 잡혔습니다. 이 경우에도 경매절차를 정지할 수 있는 방법이 있나요?  

변경된 질문:  
전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차 매각기일이 잡혔습니다. 이 경우에도 경매절차를 정지할 수 있는 방법이 있나요?  

※ 참고: 사용자 질문의 "전세사기"는 용어 사전에 '전세피해'로 직접 매핑되지 않으나, 가장 근접한 의미인 '권리리스크'로 변환하였습니다. 다만, 정확한 법률 용어 적용을 위해 추가 확인이 필요할 수 있습니다.  

(최종 출력 시 위 주석 제외)  

**최종 출력:**  
전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차 매각기일이 잡혔습니다. 이 경우에도 경매절차를 정지할 수 있는 방법이 있나요?
2026-02-03 17:12:33,763 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차 매각기일이 잡혔습니다. 이 경우에도 경매절차를 정지할 수 있는 방법이 있나요?  

변경된 질문:  
전세피해(권리리스크)로 아직 공식 결정 전인데 경매절차 매각기일이 잡혔습니다. 이 경우에도 경매절차를 정지할 수 있는 방법이 있나요?  

※ 참고: 사용자 질문의 "전세사기"는 용어 사전에 '전세피해'로 직접 매핑되지 않으나, 가장 근접한 의미인 '권리리스크'로 변환하였습니다. 다만, 정확한 법률 용어 적용을 위해 추가 확인이 필요할 수 있습니다.  

(최종 출력 시 위 주석 제외)

NameError: name 'validate_data' is not defined

# 5. 결과 정리

In [6]:
# wide CSV
wide_csv = build_wide_csv(results[0]["run_tag"], results, n_questions=len(items))


# 평균 이미지
bar_path, tbl_path = make_mean_images(results[0]["run_tag"], results, n_questions=len(items))
print("✅ MEAN images:", bar_path, tbl_path)


# failure_cases 생성
import pandas as pd


def build_failure_cases(all_runs_csv, out_csv, ri_threshold=0.7):
    df = pd.read_csv(all_runs_csv)
    fail = df[(df["Ri_mean"] < ri_threshold) | (df["must_have_found"] == False)]
    fail = fail.sort_values("Ri_mean")
    fail.to_csv(out_csv, index=False, encoding="utf-8")
    print("✅ saved failure_cases:", out_csv)


build_failure_cases(
    all_runs_csv="./data/RAGAS/all_runs.csv",
    out_csv="./data/RAGAS/failure_cases.csv",
    ri_threshold=0.70
)

NameError: name 'results' is not defined